**Предисловие:**

**Реализация основана на работе:** ...

**Результаты данной работы представлены в статье (при использовании просьба ссылаться на следующую работу):** ...

**Выборки данных:**
>*    Подписи к изображениям на английском языке из выборки COCO Image Captions
*    Стихи на русском языке с сайта stihi.ru

**Описание работы:**
> В данной работе проведены реализация, обучение и тестирование нейронных сетей для генерации случайных коротких текстов. Используются такие нейронные сети как LSTM и SeqGAN. Проведена оценка качества генерации текста на основе метрики BLEU. Для обучения и тестирования используются следующие выборки данных:
*   Подписи к изображениям на английском языке из выборки COCO Image Captions
*   Стихи на русском языке с сайта stihi.ru




Список всех установленных библиотек и их версии

In [ ]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.3.0          
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.2.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.8.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.2.1          
blis                          0.4.

Подключение к Google Drive

In [ ]:
#подключение к гугл диску
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

path = '/content/drive/My Drive/SeqGAN/'
os.chdir(path)

Объявление параметров по умолчанию

In [ ]:
from __future__ import print_function
from math import ceil
import numpy as np
import sys

import torch
import torch.optim as optim
import torch.nn as nn

import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn.init as init

import helpers # перетащить функции в код

DATASET = 'stihi_ru' # название выборки данных список выборок: ['stihi_ru', 'coco_image_captions']
CUDA = True # использование cuda (gpu/tpu): True/False
BATCH_SIZE = 1000 # количество примеров в партии/батче
MLE_TRAIN_EPOCHS = 20 # количество эпох при обучении генератора на основе MLE
DIS_TRAIN_ITERATIONS = 250 # количество итераций при обучении дискриминатора (итерации содержит несколько эпох, на каждой итерации данные для обучения обновляются)
DIS_TRAIN_EPOCHS = 1 # количество эпох в одной итерации обучения дискриминатора
ADV_TRAIN_EPOCHS = 100 # количество эпох при обучении генератора на основе соревновательного обучения / обучения с подкреплением
POS_NEG_SAMPLES = 10000

GEN_EMBEDDING_DIM = 150 # ширина слоя Embedding генератора
GEN_HIDDEN_DIM = 150 # количество нейронов в слоях генератора
DIS_EMBEDDING_DIM = 150 # ширина слоя Embedding дискриминатора
DIS_HIDDEN_DIM = 150 # количество нейронов в слоях дискриминатора

# параметры START_LETTER, VOCAB_SIZE, MAX_SEQ_LEN и FILE_PATHS задаются автоматически ниже
START_LETTER = None #! стартовое слово (слово которое подается первым на нейронную сеть генератор)
MAX_SEQ_LEN = None # длина генерируемого примера
VOCAB_SIZE = None # количество слов в словаре
FILE_PATHS = None # пути к файлам набора данных
CLOSING_WORD = None # заключительное слово / слово заполнитель ставящееся в конце предложения

# выбор набора данных и инициализация соответствующих глобальных параметров
if DATASET == 'stihi_ru': # выборка с русскими стихами с сайта stihi_ru (длина примера 10 слов, размер словаря 5 000 слов)
  START_LETTER = 0
  MAX_SEQ_LEN = 10
  VOCAB_SIZE = 5000
  CLOSING_WORD = 4999#4997
  FILE_PATHS = {'train': r'datasets/stihi_ru/stihi_ru_realtrain_cotra.txt', 'test': r'datasets/stihi_ru/stihi_ru_realtest_coco.txt',
                'vocab': r'datasets/stihi_ru/stihi_ru_vocab_cotra.pkl', 'saved_models': r'saved_models/stihi_ru'}
elif DATASET == 'coco_image_captions': # выборка COCO Image Captions (длина примера 20 слов, размер словаря 4980 слов)
  START_LETTER = 0
  MAX_SEQ_LEN = 20
  VOCAB_SIZE = 4838
  CLOSING_WORD = 1814
  FILE_PATHS = {'train': r'datasets/coco_image_captions/coco_image_captions_train.txt', 'test': r'datasets/coco_image_captions/coco_image_captions_test.txt',
                'vocab': r'datasets/coco_image_captions/coco_image_captions_vocab_cotra_test.pkl', 'saved_models': r'saved_models/coco_image_captions'}

Инициализация генераторов случайных чисел.

In [ ]:
# инициализация генераторов случайных чисел.
torch.random.manual_seed(25)
np.random.seed(25)
#системный рандом!!!

Объявление класса Генератора

In [ ]:
class Generator(nn.Module):

  def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False):
    super(Generator, self).__init__()
    self.hidden_dim = hidden_dim # количество элементов на скрытом слое
    self.embedding_dim = embedding_dim # размер слоя embedding
    self.max_seq_len = max_seq_len # длина генерируемых примеров
    self.vocab_size = vocab_size # размер словаря использующегося при генерации
    self.gpu = gpu # использование cuda (True/False)
    self.lstm_num_layers = 1 # количество LSTM слоев

    self.embeddings = nn.Embedding(vocab_size, embedding_dim) # объявление слоя embeddings
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=self.lstm_num_layers) # объявление LSTM слоев
    self.lstm2out = nn.Linear(hidden_dim, vocab_size) # объявление выходного слоя

  def init_hidden(self, batch_size=1):
    # инициализация состояния LSTM слоев
    h = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim)) 
    c = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim))

    if self.gpu:
        return h.cuda(), c.cuda()
    else:
        return h, c

  def forward(self, inp, hidden, c):
    """
    Embeds input and applies LSTM one token at a time (seq_len = 1)
    """
    # input dim                                             # batch_size
    emb = self.embeddings(inp)                              # batch_size x embedding_dim
    emb = emb.view(1, -1, self.embedding_dim)               # 1 x batch_size x embedding_dim
    out, (hidden, c) = self.lstm(emb, (hidden, c))                    # 1 x batch_size x hidden_dim (out)
    out = self.lstm2out(out.view(-1, self.hidden_dim))       # batch_size x vocab_size
    out = F.log_softmax(out, dim=1)
    return out, hidden, c

  def sample(self, num_samples, start_letter=0, degree=1):
    """
    Samples the network and returns num_samples samples of length max_seq_len.

    Outputs: samples, hidden
        - samples: num_samples x max_seq_length (a sampled sequence in each row)
    """

    samples = torch.zeros(num_samples, self.max_seq_len).type(torch.LongTensor)

    h, c = self.init_hidden(num_samples)
    inp = autograd.Variable(torch.LongTensor([start_letter]*num_samples))
    if self.gpu:
        samples = samples.cuda()
        inp = inp.cuda()

    for i in range(self.max_seq_len):
        out, h, c = self.forward(inp, h, c)               # out: num_samples x vocab_size
        out = torch.exp(out)**degree
        out = torch.multinomial(out, 1)  # num_samples x 1 (sampling from each row)
        samples[:, i] = out.view(-1).data

        inp = out.view(-1)

    return samples

  def batchNLLLoss(self, inp, target):
    """
    Returns the NLL Loss for predicting target sequence.

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len

        inp should be target with <s> (start letter) prepended
    """

    loss_fn = nn.NLLLoss()
    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)           # seq_len x batch_size
    target = target.permute(1, 0)     # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        loss += loss_fn(out, target[i])

    return loss     # per batch

  def batchPGLoss(self, inp, target, reward):
    """
    Returns a pseudo-loss that gives corresponding policy gradients (on calling .backward()).
    Inspired by the example in http://karpathy.github.io/2016/05/31/rl/

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len
        - reward: batch_size (discriminator reward for each sentence, applied to each token of the corresponding
                  sentence)

        inp should be target with <s> (start letter) prepended
    """

    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)          # seq_len x batch_size
    target = target.permute(1, 0)    # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        # TODO: should h be detached from graph (.detach())?
        for j in range(batch_size):
            loss += -out[j][target.data[i][j]]*reward[j]#     # log(P(y_t|Y_1:Y_{t-1})) * Q

    return loss/batch_size


out      0.8; 0.7; 0.1
real_out 0; 0; 1
target   2

t 
j 1 2     3      4
  Я пошел гулять . 1
  Я хочу есть. 1
  Я я я я я. 0
  0,7
Я       0
ты      1
вы      2
пошел   3
гулять  4
.       5

In [ ]:
0.5; 0.2; 0.1
70%; 30%; 20%

**1.5
0.49; 0.04; 0.01
90%; 7%; 3%

Объявяление класса Дискриминатора

In [ ]:
class Discriminator(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False, dropout=0.2):
        super(Discriminator, self).__init__()
        self.hidden_dim = hidden_dim # размер скрытого слоя
        self.embedding_dim = embedding_dim # ширина слоя embedding
        self.max_seq_len = max_seq_len # длина входного примера
        self.gpu = gpu # использование cuda (True/False)

        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # объявление слоя Embedding
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=3, bidirectional=True, dropout=dropout) # объявление LSTM слоев

        self.lstm2hidden = nn.Linear(2*3*hidden_dim, hidden_dim) # объявление скрытого слоя
        # self.lstm2hidden = nn.Linear(max_seq_len*hidden_dim*2, hidden_dim) # объявление скрытого слоя

        self.dropout_linear = nn.Dropout(p=dropout)
        self.hidden2out = nn.Linear(hidden_dim, 1) # объявление выходного слоя

    def init_hidden(self, batch_size):
        # инициализация LSTM слоев
        h = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))
        c = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))

        if self.gpu:
            return h.cuda(), c.cuda()
        else:
            return h, c

    def forward(self, input, hidden, c):
        # input dim                                                # batch_size x seq_len
        # print(input.shape)
        emb = self.embeddings(input)                               # batch_size x seq_len x embedding_dim
        # print(emb.shape)
        emb = emb.permute(1, 0, 2)                                 # seq_len x batch_size x embedding_dim
        # print(emb.shape)
        out_lstm, (hidden, c) = self.lstm(emb, (hidden, c))                          # 4 x batch_size x hidden_dim

        hidden = hidden.permute(1, 0, 2).contiguous()              # batch_size x 4 x hidden_dim
        # hidden = out_lstm
        # print(hidden.shape, hidden.view(-1, self.max_seq_len*self.hidden_dim).shape)
        # print(hidden.permute(1, 0, 2).contiguous().shape, hidden.permute(1, 0, 2).contiguous().view(-1, self.max_seq_len*self.hidden_dim*2).shape)

        out = self.lstm2hidden(hidden.view(-1, 6*self.hidden_dim))  # batch_size x 4*hidden_dim
        # out = self.lstm2hidden(hidden.view(-1, self.max_seq_len*self.hidden_dim*2))  # batch_size x 4*hidden_dim
        
        out = torch.relu(out)
        # out = out * torch.sigmoid(0.1 * out) # функция активации Swish: x * sigmoid(b*x)
        out = self.dropout_linear(out)
        out = self.hidden2out(out)                                 # batch_size x 1
        out = torch.sigmoid(out)
        return out

    def batchClassify(self, inp):
        """
        Classifies a batch of sequences.

        Inputs: inp
            - inp: batch_size x seq_len

        Returns: out
            - out: batch_size ([0,1] score)
        """

        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return out.view(-1)

    def batchBCELoss(self, inp, target):
        """
        Returns Binary Cross Entropy Loss for discriminator.

         Inputs: inp, target
            - inp: batch_size x seq_len
            - target: batch_size (binary 1/0)
        """

        loss_fn = nn.BCELoss()
        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return loss_fn(out, target)

Объявление функций возвращающих данные

In [ ]:
# функция генерации реалиных данных
def sampler_example(batch_size):
  x = data_file_train[np.random.randint(0, len(data_file_train), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

# функция генерации реалиных данных
def sampler_example_test(batch_size):
  x = data_file_test[np.random.randint(0, len(data_file_test), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

Функции обучения генератора

In [ ]:
# функция обучения генератора на основе MLE
def train_generator_MLE(gen, gen_opt, real_samples_train, real_samples_test, epochs):
    """
    Max Likelihood Pretraining for the generator
    """
    for epoch in range(epochs):
        print('epoch %d : ' % (epoch + 1), end='')
        sys.stdout.flush()
        total_loss = 0

        # обучение
        for i in range(0, len(real_samples_train), BATCH_SIZE):
            inp_train, target_train = helpers.prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                          gpu=CUDA)
            gen_opt.zero_grad()
            loss = gen.batchNLLLoss(inp_train, target_train)
            loss.backward()
            gen_opt.step()

            total_loss += loss.data.item()

            if (i / BATCH_SIZE) % ceil(
                            ceil(len(real_samples_train) / float(BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                print('.', end='')
                sys.stdout.flush()

        # each loss in a batch is loss per sample
        total_loss = total_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_train_NLL = %.4f' % total_loss, end='')

        # тестирование
        test_loss = 0

        for i in range(0, len(real_samples_test), BATCH_SIZE):
            inp_test, target_test = helpers.prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                              gpu=CUDA)
            loss = gen.batchNLLLoss(inp_test, target_test)
            test_loss += loss.data.item()
        test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_test_NLL = %.4f' % test_loss)

def test_mle(gen, real_samples_train, real_samples_test):
  '''
  Тестирование генератора на обучающей и тестовой выборках.
  '''
  # тестирование на обучающей
  test_loss = 0
  for i in range(0, len(real_samples_train), BATCH_SIZE):
      inp_test, target_test = helpers.prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print('average_train_NLL = %.4f' % test_loss, end='')

  # тестирование на тестовой
  test_loss = 0
  for i in range(0, len(real_samples_test), BATCH_SIZE):
      inp_test, target_test = helpers.prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print(' average_test_NLL = %.4f' % test_loss)

# функция обучения генератора на основе обучения с подкреплением RL
def train_generator_PG(gen, gen_opt, dis, num_batches): # !переименовать!
    """
    The generator is trained using policy gradients, using the reward from the discriminator.
    Training is done for num_batches batches.
    """

    for batch in range(num_batches):
        s = gen.sample(BATCH_SIZE*4)        # 64 works best
        inp, target = helpers.prepare_generator_batch(s, start_letter=START_LETTER, gpu=CUDA)
        rewards = dis.batchClassify(target)

        gen_opt.zero_grad()
        pg_loss = gen.batchPGLoss(inp, target, rewards)
        pg_loss.backward()
        gen_opt.step()
    print()

Функция обучения дискриминатора

In [ ]:
# функция обучения дискриминатора
def train_discriminator(discriminator, dis_opt, real_data_samples, generator, d_steps, epochs):
    """
    Training the discriminator on real_data_samples (positive) and generated samples from generator (negative).
    Samples are drawn d_steps times, and the discriminator is trained for epochs epochs.
    """

    # generating a small validation set before training
    pos_val = real_data_samples[np.random.randint(0, len(real_data_samples), 500)] #sampler_example(250)
    neg_val = generator.sample(500)
    val_inp, val_target = helpers.prepare_discriminator_data(pos_val, neg_val, gpu=CUDA)

    for d_step in range(d_steps):
        s = helpers.batchwise_sample(generator, POS_NEG_SAMPLES, BATCH_SIZE)
        dis_inp, dis_target = helpers.prepare_discriminator_data(real_data_samples, s, gpu=CUDA)
        val_pred = discriminator.batchClassify(val_inp)
        print('ДО ОБУЧЕНИЯ: val_acc = %.4f' % (
            torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))
        for epoch in range(epochs):
            print('d-step %d epoch %d : ' % (d_step + 1, epoch + 1), end='')
            sys.stdout.flush()
            total_loss = 0
            total_acc = 0

            for i in range(0, 2 * POS_NEG_SAMPLES, BATCH_SIZE):
                inp, target = dis_inp[i:i + BATCH_SIZE], dis_target[i:i + BATCH_SIZE]
                dis_opt.zero_grad()
                out = discriminator.batchClassify(inp)
                loss_fn = nn.BCELoss()
                loss = loss_fn(out, target)
                loss.backward()
                dis_opt.step()

                total_loss += loss.data.item()
                total_acc += torch.sum((out>0.5)==(target>0.5)).data.item()

                if (i / BATCH_SIZE) % ceil(ceil(2 * POS_NEG_SAMPLES / float(
                        BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                    print('.', end='')
                    sys.stdout.flush()

            total_loss /= ceil(2 * POS_NEG_SAMPLES / float(BATCH_SIZE))
            total_acc /= float(2 * POS_NEG_SAMPLES)

            val_pred = discriminator.batchClassify(val_inp)
            print(' average_loss = %.4f, train_acc = %.4f, val_acc = %.4f' % (
                total_loss, total_acc, torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))

Функция оценки качества генерации текста

In [ ]:
# оценка качества по BLEU метрике
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
from datetime import datetime
import nltk
import random
from scipy import stats

# функция оценки качества генерации текста по метрике BLEU
def BLEU(reference_sample, test_sample, print_iteration=100):
  print("--- --- ---\nStart BLEU")
  pad = CLOSING_WORD
  #################################################
  reference = []
  for line in reference_sample:
    candidate = []
    for i in line:
      if i == pad:
        break
      candidate.append(i)

    reference.append(candidate)
  #################################################
  hypothesis_list_leakgan = []
  for line in test_sample:
    while line[-1] == str(pad):
      line.remove(str(pad))
    hypothesis_list_leakgan.append(line)
  #################################################
  random.shuffle(hypothesis_list_leakgan)
  #################################################

  smoothing_function = SmoothingFunction().method1

  for ngram in range(2,6):
      weight = tuple((1. / ngram for _ in range(ngram)))
      bleu_leakgan = []
      bleu_supervise = []
      bleu_base2 = []
      num = 0
      for h in hypothesis_list_leakgan:
          BLEUscore = nltk.translate.bleu_score.sentence_bleu(reference, h, weight, smoothing_function = smoothing_function)
          num += 1
          bleu_leakgan.append(BLEUscore)

          if num%print_iteration == 0:
            print(ngram, num, sum(bleu_leakgan)/len(bleu_leakgan))
      print ('--- --- ---')
      print(len(weight), '-gram BLEU score : ', 1.0 * sum(bleu_leakgan) / len(bleu_leakgan), "\n")

Функция сохранения моделей

In [ ]:
# функция сохранения моделей (сериализация: моделей генератора и дискриминатора, параметров по умолчанию, обучающих данных)
def save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer, name):
  state = {
      'default_parameters': {'VOCAB_SIZE': VOCAB_SIZE, 'MAX_SEQ_LEN': MAX_SEQ_LEN, 'GEN_EMBEDDING_DIM': GEN_EMBEDDING_DIM,
                             'GEN_HIDDEN_DIM': GEN_HIDDEN_DIM, 'DIS_EMBEDDING_DIM': DIS_EMBEDDING_DIM, 'DIS_HIDDEN_DIM': DIS_HIDDEN_DIM},
      'data_file_tensor_train': data_file_tensor_train,
      'gen_state_dict': gen.state_dict(),
      'dis_state_dict': dis.state_dict(),
      'gen_optimizer': gen_optimizer.state_dict(),
      'dis_optimizer': dis_optimizer.state_dict(),
  }
  torch.save(state, name)

Функция загрузки моделей

In [ ]:
# функция загрузки моделей (десериализация: моделей генератора и дискриминатора, параметров по умолчанию, обучающих данных)
def load_models(name):
  if CUDA:
    device = torch.device('cuda')
  else:
    device = torch.device('cpu')

  print('state')
  state = torch.load(name, map_location=device)

  print('default_parameters')
  VOCAB_SIZE = state['default_parameters']['VOCAB_SIZE']
  MAX_SEQ_LEN = state['default_parameters']['MAX_SEQ_LEN']
  GEN_EMBEDDING_DIM = state['default_parameters']['GEN_EMBEDDING_DIM']
  GEN_HIDDEN_DIM = state['default_parameters']['GEN_HIDDEN_DIM']
  DIS_EMBEDDING_DIM = state['default_parameters']['DIS_EMBEDDING_DIM']
  DIS_HIDDEN_DIM = state['default_parameters']['DIS_HIDDEN_DIM']

  print('data_file_tensor_train')
  data_file_tensor_train = torch.tensor(state['data_file_tensor_train'])

  print('Generator')
  gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  gen.load_state_dict(state['gen_state_dict'])
  gen_optimizer = optim.Adam(gen.parameters(), lr=0.001)
  gen_optimizer.load_state_dict(state['gen_optimizer'])

  print('Discriminator')
  dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  dis.load_state_dict(state['dis_state_dict'])
  dis_optimizer = optim.Adagrad(dis.parameters())
  dis_optimizer.load_state_dict(state['dis_optimizer'])

  print('CUDA')
  if CUDA:
    data_file_tensor_train = data_file_tensor_train.cuda()
    gen = gen.cuda()
    dis = dis.cuda()
  
  return [data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
          VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM]

Загрузка набора данных

In [ ]:
# загрузка словаря
import pickle

vocab_file = FILE_PATHS['vocab']
word, vocab = pickle.load(open(vocab_file, 'rb'))

In [ ]:
# загрузка обучающей выборки
f = open(FILE_PATHS['train'], 'r')
data_file_train = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_train.append(line)
data_file_train = np.array(data_file_train)[:, :MAX_SEQ_LEN]
print("Примеров в обучающей выборке: ", len(data_file_train))

Примеров в обучающей выборке:  70000


In [ ]:
# загрузка тестовой выборки
f = open(FILE_PATHS['test'], 'r')
data_file_test = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_test.append(line)
data_file_test = np.array(data_file_test)[:, :MAX_SEQ_LEN]
print("Примеров в тестовой выборке: ", len(data_file_test))

Примеров в тестовой выборке:  5000


In [ ]:
# примеры из обучающей выборки
print("Примеры из обучающей выборки")
samples = sampler_example(50)[0]
output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line) + '\n'
  output_function.append(line)

for i, output in enumerate(output_function):
  print("#", i, "\tПример: ", output)    

Примеры из обучающей выборки
# 0 	Пример:  по ходу знаешь нам уже давным давно с тобой пора

# 1 	Пример:  я люблю а значит знаю не могу я без тебя

# 2 	Пример:  пусть всё как было там останется в тиши  

# 3 	Пример:  той что так и рвётся сейчас из под кожи 

# 4 	Пример:  шли в гору с молитвой христа и глубин  

# 5 	Пример:  ну а ты никого уже больше не ждёшь  

# 6 	Пример:  себя богу отдам не жалея об этом ничуть  

# 7 	Пример:  моя любовь до гроба моя любовь навечно но  

# 8 	Пример:  те кто умеет ждать те кто умеет прощать  

# 9 	Пример:  у каждого из нас в душе к нему дорога 

# 10 	Пример:  что не смог быть с тобой до конца  

# 11 	Пример:  мне жаль что я так и не смог  

# 12 	Пример:  моя маленькая добрая вера всё понимает и больше молчит 

# 13 	Пример:  но вот забыл мне до чего здесь нужен  

# 14 	Пример:  я в ответ скажу что мне ближе космос  

# 15 	Пример:  будто я понимаю что все это не то  

# 16 	Пример:  всё я вспомнил тогда и подумал а мне  

# 17 	Пример:  б

Оценка качества примеров из обучающей выборки на основе BLEU  (для сравнения с генератором)

In [ ]:
# примеры сгенерированных текстов
print("Примеры реальных текстов\n")
samples = sampler_example(50)[0]

output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line) + '\n'
  # output_function.append(line)
  print("#", "\tПример: ", line, end='')    
  BLEU(data_file_test.tolist(), [samp])

# for i, output in enumerate(output_function):
#   print("#", i, "\tПример: ", output, end='')

Примеры реальных текстов

# 	Пример:  пусть в душе моей с каждым днем все сильнее грусть
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  0.7453559924999299 

--- --- ---
3 -gram BLEU score :  0.5178720843256431 

--- --- ---
4 -gram BLEU score :  0.21105340631872638 

--- --- ---
5 -gram BLEU score :  0.12702337351164256 

# 	Пример:  скажи мне хоть что нибудь милый мой друг  
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  0.816496580927726 

--- --- ---
3 -gram BLEU score :  0.6299605249474366 

--- --- ---
4 -gram BLEU score :  0.24446151121745052 

--- --- ---
5 -gram BLEU score :  0.1428690456354165 

# 	Пример:  от того что я с тобой в раю  
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  1.0 

--- --- ---
3 -gram BLEU score :  0.9564655913861946 

--- --- ---
4 -gram BLEU score :  0.7071067811865475 

--- --- ---
5 -gram BLEU score :  0.3341625309791344 

# 	Пример:  твой мир подходит к моему он такой светлый  
--- --- ---
Start BLEU
--- --- ---
2 -gra

In [ ]:
%%time
# оценка качества на основе примеров из обучающей выборки (для сравнения с генератором)
print("Оценка примеров из обучающей выборки на основе BLEU")
BLEU(data_file_test.tolist(), data_file_train[:500].tolist(), print_iteration=100)

'''
BLEU примеров из обучающей выборки
--- --- ---
Start BLEU
2 100 0.5365504958995001
2 200 0.5426700180034696
2 300 0.5425210147596119
2 400 0.5438925313146432
2 500 0.5499117864293879
--- --- ---
2 -gram BLEU score :  0.5499117864293879
3 100 0.4252853298296578
3 200 0.4294441261529973
3 300 0.42938411299670276
3 400 0.432946142036555
3 500 0.436991427933691
--- --- ---
3 -gram BLEU score :  0.436991427933691
4 100 0.3118228221158724
4 200 0.31347352868277367
4 300 0.3133847969805193
4 400 0.3148005592801212
4 500 0.3173224414085976
--- --- ---
4 -gram BLEU score :  0.3173224414085976
5 100 0.20748007111715946
5 200 0.21057918753505803
5 300 0.20884027925072032
5 400 0.20986772415037908
5 500 0.21083373415979106
--- --- ---
5 -gram BLEU score :  0.21083373415979106
'''

Оценка примеров из обучающей выборки на основе BLEU
--- --- ---
Start BLEU
2 100 0.7383136866999611
2 200 0.7434567933758077
2 300 0.7372338773276429
2 400 0.7399847238118723
2 500 0.7379809479854453
--- --- ---
2 -gram BLEU score :  0.7379809479854453 

3 100 0.4269311649127271
3 200 0.4303715351026588
3 300 0.4226227528288037
3 400 0.42873107916079045
3 500 0.42543848729360106
--- --- ---
3 -gram BLEU score :  0.42543848729360106 

4 100 0.22424143250767756
4 200 0.22453770989776117
4 300 0.2211325190507896
4 400 0.2253970685457768
4 500 0.2242617727684013
--- --- ---
4 -gram BLEU score :  0.2242617727684013 

5 100 0.1366408381390683
5 200 0.13738905184583203
5 300 0.13432237449273152
5 400 0.13652256802730714
5 500 0.1362537152585565
--- --- ---
5 -gram BLEU score :  0.1362537152585565 

CPU times: user 5min 55s, sys: 46.5 ms, total: 5min 55s
Wall time: 5min 55s


Создание нейронных сетей генератора и дискриминатора

In [ ]:
# объявление нейронных сетей генератора и дискриминатора, подготовка выборок данных для использования pytorch
gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)

if CUDA:
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()
else:
  data_file_tensor_train = torch.tensor(data_file_train)
  data_file_tensor_test = torch.tensor(data_file_test)

gen_optimizer = optim.Adam(gen.parameters(), lr=0.001) #, lr=0.001
dis_optimizer = optim.Adagrad(dis.parameters())

Обучение генератора на основе MLE

In [ ]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 8.5058 average_test_NLL = 8.5068
CPU times: user 1.76 s, sys: 147 ms, total: 1.91 s
Wall time: 2.03 s


In [ ]:
%%time
# обучение генератора на основе MLE / предобучение генератора
print('Запуск обучения генератора на основе MLE...')
gen_optimizer = optim.Adam(gen.parameters())#, lr=0.0002
train_generator_MLE(gen, gen_optimizer, data_file_tensor_train, data_file_tensor_test, 5) # MLE_TRAIN_EPOCHS

test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_mle.pytorch')
# epoch 1 : .......... average_train_NLL = 1.8447 average_test_NLL = 1.9937

Запуск обучения генератора на основе MLE...
epoch 1 : .......... average_train_NLL = 4.0814 average_test_NLL = 4.4963
epoch 2 : .......... average_train_NLL = 4.0588 average_test_NLL = 4.4956
epoch 3 : .......... average_train_NLL = 4.0447 average_test_NLL = 4.4933
epoch 4 : .......... average_train_NLL = 4.0293 average_test_NLL = 4.4929
epoch 5 : .......... average_train_NLL = 4.0145 average_test_NLL = 4.4934
average_train_NLL = 3.9923 average_test_NLL = 4.4934
CPU times: user 9.32 s, sys: 2.87 s, total: 12.2 s
Wall time: 12.5 s


In [ ]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 3.9923 average_test_NLL = 4.4934
CPU times: user 654 ms, sys: 137 ms, total: 791 ms
Wall time: 796 ms


Генерация примеров текстов на основе MLE

In [ ]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе MLE\n")
samples = gen.sample(50, degree=1).cpu().detach().numpy()

output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line) + '\n'
  # output_function.append(line)
  print("#", "\tПример: ", line, end='')    
  BLEU(data_file_test.tolist(), [samp])

# for i, output in enumerate(output_function):
#   print("#", i, "\tПример: ", output, end='')

Примеры генерируемых текстов на основе MLE

# 	Пример:  я дам очень много лет нет чуть чуть но мама
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  0.6666666666666666 

--- --- ---
3 -gram BLEU score :  0.17710976153043526 

--- --- ---
4 -gram BLEU score :  0.09438595268231116 

--- --- ---
5 -gram BLEU score :  0.06672608446749886 

# 	Пример:  во тьме ночной в жизни мы такие разные  
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  0.6666666666666666 

--- --- ---
3 -gram BLEU score :  0.381571414184444 

--- --- ---
4 -gram BLEU score :  0.167844596251862 

--- --- ---
5 -gram BLEU score :  0.10575371703212418 

# 	Пример:  ты не знал какой не плакал ничего потому что б
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  0.7453559924999299 

--- --- ---
3 -gram BLEU score :  0.5178720843256431 

--- --- ---
4 -gram BLEU score :  0.21105340631872638 

--- --- ---
5 -gram BLEU score :  0.12702337351164256 

# 	Пример:  она хочет иметь того что есть в самом  
---

Оценка качества генерации текста после обучения с помощью MLE

In [ ]:
%%time
print("Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE")
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE
--- --- ---
Start BLEU
2 100 0.7014587652128759
2 200 0.7039079259301146
2 300 0.7021673762378553
2 400 0.7030312526563669
2 500 0.709756420006514
--- --- ---
2 -gram BLEU score :  0.709756420006514 

3 100 0.3591994149424474
3 200 0.37195198057311624
3 300 0.3630959184724861
3 400 0.36953015744616735
3 500 0.3753246389420084
--- --- ---
3 -gram BLEU score :  0.3753246389420084 

4 100 0.18462732454484967
4 200 0.18893176285733237
4 300 0.18428882293739865
4 400 0.1858392282817685
4 500 0.18765159960056088
--- --- ---
4 -gram BLEU score :  0.18765159960056088 

5 100 0.11279694351365704
5 200 0.11861471052834853
5 300 0.11513113747064646
5 400 0.11533883980791282
5 500 0.11604495759756471
--- --- ---
5 -gram BLEU score :  0.11604495759756471 

CPU times: user 6min 7s, sys: 99.8 ms, total: 6min 7s
Wall time: 6min 7s


Предварительное обучение дискриминатора

In [ ]:
%%time
# предобучение дискриминатора
print('Запуск обучения дискриминатора...')
# dis_optimizer = optim.Adagrad(dis.parameters()) # , lr=0.0001
# dis_optimizer = optim.Adam(gen.parameters(), lr=0.001, weight_decay=1e-5)#0.001
# dis_optimizer = optim.Adadelta(gen.parameters())
# dis_optimizer = optim.Adagrad(dis.parameters())#, lr=0.0001)#, weight_decay=1e-5)
#, weight_decay=1e-5) # регуляризация
train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, DIS_TRAIN_ITERATIONS, DIS_TRAIN_EPOCHS)#25, 1 | (15, 3), (25, 1)

# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_pretraining_dis.pytorch')

# ДО ОБУЧЕНИЯ: val_acc = 0.5230
# d-step 50 epoch 1 : .......... average_loss = 0.2859, train_acc = 0.9136, val_acc = 0.5240

Запуск обучения дискриминатора...
ДО ОБУЧЕНИЯ: val_acc = 0.8520
d-step 1 epoch 1 : .......... average_loss = 0.1648, train_acc = 0.9553, val_acc = 0.8400
ДО ОБУЧЕНИЯ: val_acc = 0.8380
d-step 2 epoch 1 : .......... average_loss = 0.1618, train_acc = 0.9566, val_acc = 0.8500
ДО ОБУЧЕНИЯ: val_acc = 0.8520
d-step 3 epoch 1 : .......... average_loss = 0.1613, train_acc = 0.9564, val_acc = 0.8370
ДО ОБУЧЕНИЯ: val_acc = 0.8400
d-step 4 epoch 1 : .......... average_loss = 0.1623, train_acc = 0.9572, val_acc = 0.8300
ДО ОБУЧЕНИЯ: val_acc = 0.8210
d-step 5 epoch 1 : .......... average_loss = 0.1671, train_acc = 0.9547, val_acc = 0.8480
ДО ОБУЧЕНИЯ: val_acc = 0.8490
d-step 6 epoch 1 : .......... average_loss = 0.1611, train_acc = 0.9572, val_acc = 0.8080
ДО ОБУЧЕНИЯ: val_acc = 0.8090
d-step 7 epoch 1 : .......... average_loss = 0.1635, train_acc = 0.9556, val_acc = 0.8540
ДО ОБУЧЕНИЯ: val_acc = 0.8560
d-step 8 epoch 1 : .......... average_loss = 0.1682, train_acc = 0.9556, val_acc = 0.8340
ДО ОБУ

Оценка качества генерации на основе выбора дискриминатором наиболее качественных текстов

In [ ]:
# генерация примеров на основе отбора искриминатором
print("Generation: ", end='')
sample_pre_dis = []
for i in range(500):
  samples = gen.sample(10)
  out_dis = dis.batchClassify(samples).cpu().detach().numpy()
  samp = samples[np.argmax(out_dis)]
  sample_pre_dis.append(samp.tolist())

  if(i%50==0):
    print(end='.')

Generation: ..........

In [ ]:
BLEU(data_file_test.tolist(), sample_pre_dis[:500], print_iteration=100)

--- --- ---
Start BLEU
2 100 0.5216714542547094
2 200 0.5297161381445916
2 300 0.5231139981214722
2 400 0.5223088514427205
2 500 0.5254618616659142
--- --- ---
2 -gram BLEU score :  0.5254618616659142 

3 100 0.4108526764211435
3 200 0.4137936399143692
3 300 0.40741745886267894
3 400 0.4063092944221004
3 500 0.40914688066322547
--- --- ---
3 -gram BLEU score :  0.40914688066322547 

4 100 0.2979217342618846
4 200 0.296667617120298
4 300 0.288269431342115
4 400 0.2872150644540361
4 500 0.29029748519890014
--- --- ---
4 -gram BLEU score :  0.29029748519890014 

5 100 0.20867213526948777
5 200 0.20258659528770168
5 300 0.19237883190056357
5 400 0.19173801987690792
5 500 0.1949146454075196
--- --- ---
5 -gram BLEU score :  0.1949146454075196 



Состязательное обучение. Обучение генератора на основе обучения с подкреплением

In [ ]:
test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# average_train_NLL = 1.7516 average_test_NLL = 1.9832

average_train_NLL = 3.9923 average_test_NLL = 4.4934


In [ ]:
%%time
# gen_optimizer = optim.Adagrad(gen.parameters(), lr=0.0005)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters())
# gen_optimizer = optim.Adam(gen.parameters(), lr=0.0001)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters(), lr=0.001)#, lr=0.002

# состязательное обучение генератора
print('\nStarting Adversarial Training...')

for epoch in range(25):# ADV_TRAIN_EPOCHS
    print('\n--------\nEPOCH %d\n--------' % (epoch+1))
    # обучение генератора
    print('\nAdversarial Training Generator : ', end='')
    train_generator_PG(gen, gen_optimizer, dis, 1)#

    # тестирование nll
    print('\nTesting Generator : ', end='')
    test_mle(gen, data_file_tensor_train, data_file_tensor_test)

    # обучение дискриминатора
    print('\nAdversarial Training Discriminator : ')
    train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, 5, 1)#3, 1

# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')
# epoch 3 : .......... average_train_NLL = 1.8005 average_test_NLL = 1.9824
# average_train_NLL = 1.7340 average_test_NLL = 2.0599


Starting Adversarial Training...

--------
EPOCH 1
--------

Adversarial Training Generator : 

Testing Generator : average_train_NLL = 3.9938 average_test_NLL = 4.4935

Adversarial Training Discriminator : 
ДО ОБУЧЕНИЯ: val_acc = 0.8300
d-step 1 epoch 1 : .......... average_loss = 0.1508, train_acc = 0.9601, val_acc = 0.8300
ДО ОБУЧЕНИЯ: val_acc = 0.8290
d-step 2 epoch 1 : .......... average_loss = 0.1574, train_acc = 0.9578, val_acc = 0.8240
ДО ОБУЧЕНИЯ: val_acc = 0.8210
d-step 3 epoch 1 : .......... average_loss = 0.1595, train_acc = 0.9575, val_acc = 0.8410
ДО ОБУЧЕНИЯ: val_acc = 0.8380
d-step 4 epoch 1 : .......... average_loss = 0.1510, train_acc = 0.9607, val_acc = 0.7880
ДО ОБУЧЕНИЯ: val_acc = 0.7860
d-step 5 epoch 1 : .......... average_loss = 0.1497, train_acc = 0.9603, val_acc = 0.8420

--------
EPOCH 2
--------

Adversarial Training Generator : 

Testing Generator : average_train_NLL = 3.9962 average_test_NLL = 4.4946

Adversarial Training Discriminator : 
ДО ОБУЧЕНИЯ: val

Тестирование генератора на обучающей выборке после состязательного обучения.

In [ ]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# average_train_NLL = 1.5186 average_test_NLL = 2.0730
# average_train_NLL = 1.4459 average_test_NLL = 1.9926

average_train_NLL = 4.0618 average_test_NLL = 4.5580
CPU times: user 635 ms, sys: 185 ms, total: 820 ms
Wall time: 834 ms


Генерация примеров текстов на основе SeqGAN

In [ ]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе SeqGAN")
degree = 1.5
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line) + '\n'
  output_function.append(line)

for i, output in enumerate(output_function):
  print("#", i, "\tПример: ", output)    

Примеры генерируемых текстов на основе SeqGAN
Degree: 1.5
# 0 	Пример:  и я с собою и не спал и не хочу

# 1 	Пример:  и я вас не любил и не могу  

# 2 	Пример:  и обидно что ты не со мной ей  

# 3 	Пример:  почему же ты сам не знал и не смог 

# 4 	Пример:  и о том что ты не со мной не рядом

# 5 	Пример:  но я хочу быть с тобой в разлуке  

# 6 	Пример:  и я в себе не верю не забывай  

# 7 	Пример:  но я без тебя не могу жить нельзя  

# 8 	Пример:  и в поэзии нет ни сердца ни стыда  

# 9 	Пример:  и что я думал что то много не так 

# 10 	Пример:  и ты где то там где не найдёшь  

# 11 	Пример:  в нем есть на свете ты есть одно  

# 12 	Пример:  но снова в этом мире не так мало  

# 13 	Пример:  в этот миг в этом мире в этот час 

# 14 	Пример:  ты не знаешь как ты и ты меня не знал

# 15 	Пример:  ты не слышишь я тебя люблю и но  

# 16 	Пример:  я не верю в ответ что вам нравится  

# 17 	Пример:  я просил так чтобы в нашей жизни все не спеша

# 18 	Пример:  ведь я вам пишу пр

Оценка качества генерации на основе SeqGAN

In [ ]:
%%time
print("Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN")
degree = 1
print("Degree:", degree)
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN
Degree: 1
--- --- ---
Start BLEU
2 100 0.7920559011148306
2 200 0.7887641831826584
2 300 0.7937269163708915
2 400 0.7939591059956186
2 500 0.7984360636167993
--- --- ---
2 -gram BLEU score :  0.7984360636167993 

3 100 0.47704370749175845
3 200 0.47799840448681613
3 300 0.4843832071317479
3 400 0.4832728427921858
3 500 0.489964572201849
--- --- ---
3 -gram BLEU score :  0.489964572201849 

4 100 0.250431563464843
4 200 0.24893926120651724
4 300 0.2540276641940921
4 400 0.25528966781580276
4 500 0.2597393057538393
--- --- ---
4 -gram BLEU score :  0.2597393057538393 

5 100 0.14738963688197013
5 200 0.1449478840126272
5 300 0.14888295982800126
5 400 0.14904865269430323
5 500 0.1528338137150235
--- --- ---
5 -gram BLEU score :  0.1528338137150235 

CPU times: user 5min 43s, sys: 58.2 ms, total: 5min 43s
Wall time: 5min 43s


In [ ]:
%%time
print("Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN")
degree = 1.5
print("Degree:", degree)
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN
Degree: 1.5
--- --- ---
Start BLEU
2 100 0.9363851648649012
2 200 0.9324601752966781
2 300 0.9316161415874787
2 400 0.9348450212500775
2 500 0.9359139728690822
--- --- ---
2 -gram BLEU score :  0.9359139728690822 

3 100 0.7615601842320227
3 200 0.7522667669663021
3 300 0.7545101554669555
3 400 0.7589177703294874
3 500 0.7611256913766336
--- --- ---
3 -gram BLEU score :  0.7611256913766336 

4 100 0.49123043181778014
4 200 0.46684872916604797
4 300 0.47292500988248465
4 400 0.47883441757251993
4 500 0.4829458188650377
--- --- ---
4 -gram BLEU score :  0.4829458188650377 

5 100 0.28703447155768663
5 200 0.2734864690484696
5 300 0.2742747755699115
5 400 0.27671049594390373
5 500 0.2779209564668702
--- --- ---
5 -gram BLEU score :  0.2779209564668702 

CPU times: user 5min 39s, sys: 77.5 ms, total: 5min 39s
Wall time: 5min 39s


In [ ]:
%%time
print("Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN")
degree = 2
print("Degree:", degree)
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN
Degree: 2
--- --- ---
Start BLEU
2 100 0.9620024371088012
2 200 0.9633655420093835
2 300 0.9660268649154347
2 400 0.9670843335536563
2 500 0.9678348046710067
--- --- ---
2 -gram BLEU score :  0.9678348046710067 

3 100 0.862405781358344
3 200 0.8557159419354221
3 300 0.8612680696803156
3 400 0.8617547107789805
3 500 0.8598888095458309
--- --- ---
3 -gram BLEU score :  0.8598888095458309 

4 100 0.6128507432145099
4 200 0.6120900685964918
4 300 0.6237671041368048
4 400 0.6210427898809088
4 500 0.6181192433684115
--- --- ---
4 -gram BLEU score :  0.6181192433684115 

5 100 0.3730874408151868
5 200 0.3760388202327093
5 300 0.3905292268190749
5 400 0.3871494120407027
5 500 0.38325428096499037
--- --- ---
5 -gram BLEU score :  0.38325428096499037 

CPU times: user 5min 29s, sys: 64.6 ms, total: 5min 29s
Wall time: 5min 29s


In [ ]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе SeqGAN\n")
samples = gen.sample(50, degree=1).cpu().detach().numpy()

output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line) + '\n'
  # output_function.append(line)
  print("#", "\tПример: ", line, end='')    
  BLEU(data_file_test.tolist(), [samp])

# for i, output in enumerate(output_function):
#   print("#", i, "\tПример: ", output, end='')

Примеры генерируемых текстов на основе SeqGAN

# 	Пример:  тебя никто не забыл эти пути злые свои  
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  0.5773502691896257 

--- --- ---
3 -gram BLEU score :  0.16091489743427165 

--- --- ---
4 -gram BLEU score :  0.08783602619713964 

--- --- ---
5 -gram BLEU score :  0.0629952630233744 

# 	Пример:  а может это что я рядом но не знаю что
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  0.9428090415820634 

--- --- ---
3 -gram BLEU score :  0.7631428283688879 

--- --- ---
4 -gram BLEU score :  0.28227983861579553 

--- --- ---
5 -gram BLEU score :  0.16029266087564348 

# 	Пример:  чтоб душа в их глазах к тебе не зря 
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  0.8819171036881969 

--- --- ---
3 -gram BLEU score :  0.6631762013160654 

--- --- ---
4 -gram BLEU score :  0.2540663740773074 

--- --- ---
5 -gram BLEU score :  0.14734231924041577 

# 	Пример:  оказалось что мы все мы пришли в сердце тишь 
--- --- 

In [ ]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе MLE\n")
samples = gen.sample(50, degree=2).cpu().detach().numpy()

output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line) + '\n'
  # output_function.append(line)
  print("#", "\tПример: ", line, end='')    
  BLEU(data_file_test.tolist(), [samp])

# for i, output in enumerate(output_function):
#   print("#", i, "\tПример: ", output, end='')

Примеры генерируемых текстов на основе MLE

# 	Пример:  и что я не знаю что я живу  
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  1.0 

--- --- ---
3 -gram BLEU score :  1.0 

--- --- ---
4 -gram BLEU score :  0.8091067115702212 

--- --- ---
5 -gram BLEU score :  0.5898945623563899 

# 	Пример:  и ты не веришь в то что я  
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  1.0 

--- --- ---
3 -gram BLEU score :  0.7937005259840998 

--- --- ---
4 -gram BLEU score :  0.5169731539571706 

--- --- ---
5 -gram BLEU score :  0.26010227020433874 

# 	Пример:  и на душе я не чувствую что мне  
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  1.0 

--- --- ---
3 -gram BLEU score :  0.7937005259840998 

--- --- ---
4 -gram BLEU score :  0.29071536848410967 

--- --- ---
5 -gram BLEU score :  0.1641134377013993 

# 	Пример:  и не было бы что то не так  
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  1.0 

--- --- ---
3 -gram BLEU score :  0.9564655913861946 


In [ ]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе SeqGAN\n")
samples = gen.sample(50, degree=1).cpu().detach().numpy()

output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line) + '\n'
  # output_function.append(line)
  print("#", "\tПример: ", line, end='')    
  BLEU(data_file_test.tolist(), [samp])

# for i, output in enumerate(output_function):
#   print("#", i, "\tПример: ", output, end='')

Примеры генерируемых текстов на основе SeqGAN

# 	Пример:  ты знаешь что я совсем не хотел я  
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  0.9428090415820634 

--- --- ---
3 -gram BLEU score :  0.6933612743506347 

--- --- ---
4 -gram BLEU score :  0.2626909894424158 

--- --- ---
5 -gram BLEU score :  0.15133029928492392 

# 	Пример:  как я стараюсь уйти да как все так же 
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  0.816496580927726 

--- --- ---
3 -gram BLEU score :  0.43679023236814946 

--- --- ---
4 -gram BLEU score :  0.18575057999133604 

--- --- ---
5 -gram BLEU score :  0.11468692082056794 

# 	Пример:  в прошлом а стоны зимы то и всерьёз  
--- --- ---
Start BLEU
--- --- ---
2 -gram BLEU score :  0.6666666666666666 

--- --- ---
3 -gram BLEU score :  0.17710976153043526 

--- --- ---
4 -gram BLEU score :  0.09438595268231116 

--- --- ---
5 -gram BLEU score :  0.06672608446749886 

# 	Пример:  когда ты любишь и даже не нужен мне покоя такое
--- --- -

KeyboardInterrupt: ignored

Загрузка сохраненной модели

In [ ]:
# загрузка моделей
[data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM,
 DIS_HIDDEN_DIM] = load_models(FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')#[seqgan_mle, seqgan_pretraining_dis, seqgan_adversarial_training]

if(CUDA):
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_tensor_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()

state
default_parameters
data_file_tensor_train
Generator
Discriminator
CUDA


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


In [ ]:
# плагиат из обучающей выборки
samples = gen.sample(1500).cpu().detach().numpy().tolist()
# samples = data_file_test.tolist()[:500]
train_samples = data_file_train.tolist()
n = 0
for i in range(len(samples)):
  if samples[i] in train_samples:
    n += 1
  if i%(len(samples)//10) == 0:
    print(i/len(samples)*100, "%")
print("Совпало сгенерированных примеров с обучающей выборкой: ", n, "из", len(samples))
print("Плагиат: ", n/len(samples)*100, "%")

In [ ]:
# оригинальность примеров
N_samples = 1500
samples_1 = gen.sample(N_samples).cpu().detach().numpy().tolist()
samples_2 = gen.sample(N_samples).cpu().detach().numpy().tolist()

n = 0
for i in range(len(samples_1)):
  if samples_1[i] in samples_2:
    n += 1
  if i%(len(samples_1)//10) == 0:
    print(i/len(samples_1)*100, "%")

print("Оригинальность генерируемых примеров")
print("Совпало примеров из выборки 1 с примерами из выборки 2: ", n, "из", len(samples_1))
print("Плагиат: ", n/len(samples_1)*100, "%")
print("Оригинальность: ", (1-n/len(samples_1))*100, "%")

In [ ]:
# оригинальность примеров подход 2
N_samples = 1500
samples = gen.sample(N_samples).cpu().detach().numpy().tolist()

i = 0
n = 0
while len(samples)>0:
  if samples[0] not in samples[1:]:
    n += 1
  if i%(N_samples//10) == 0:
    print(i/N_samples*100, "%")
  del(samples[0])
  i += 1

print("Оригинальность генерируемых примеров")
print("Сгенерировано оригинальных примеров: ", n, "из", N_samples)
print("Оригинальность: ", n/N_samples*100, "%")
print("Плагиат: ", (1-n/N_samples)*100, "%")